In [18]:

import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

dataname = "polynomial_feature_regression"

DATA_PATH = "../../Datasets/Vehice dataset/Downsampled/Test/test_sampled.csv"
OUTPUT_PATH = f"../../Datasets/Evaluations/Regression/{dataname}.csv"

TEST_PATH = "../../Datasets/Vehice dataset/Downsampled/Test/test_sampled.csv"
TRAIN_PATH = "../../Datasets/Vehice dataset/Downsampled/Train/train.csv"
VALID_PATH = "../../Datasets/Vehice dataset/Downsampled/Valid/valid.csv"




In [9]:
df_samples = pd.read_csv(DATA_PATH)
df_test = pd.read_csv(DATA_PATH)
df_train = pd.read_csv(TRAIN_PATH)

df_test.drop(columns=['prediction', 'prompt'], inplace=True)
df_samples.drop(columns=['prediction', 'prompt'], inplace=True)
print(df_samples.head())

                                    name  year  selling_price  km_driven  \
0     Ford Figo Aspire 1.5 TDCi Titanium  2017         670000      70000   
1  Mahindra Scorpio VLX 2WD AIRBAG BSIII  2012         525000     120000   
2                 Maruti Swift Dzire VDI  2014         438999      81000   
3              Ford Figo Diesel Titanium  2010         144000      50000   
4                 Hyundai i10 Magna 1.1L  2008         185000     110000   

     fuel seller_type transmission                 owner     mileage   engine  \
0  Diesel  Individual       Manual           First Owner  25.83 kmpl  1498 CC   
1  Diesel  Individual       Manual           First Owner  12.05 kmpl  2179 CC   
2  Diesel      Dealer       Manual           First Owner   23.4 kmpl  1248 CC   
3  Diesel  Individual       Manual          Second Owner   20.0 kmpl  1399 CC   
4  Petrol  Individual       Manual  Fourth & Above Owner  19.81 kmpl  1086 CC   

   max_power               torque  seats  
0     99 bhp 

In [10]:

df_samples = df_samples.dropna(how='any')

df_samples['prediction'] = None
df_samples["prompt"] = f'{dataname}'
df_samples["prompt_name"] = f'{dataname}'
df_samples["model_name"] = f'{dataname}'


In [11]:
def pre_process(df):
    df['year'] = 2020 - df['year']
    df['mileage'] = df['mileage'].str.replace(' kmpl', '', regex=False)
    df['mileage'] = df['mileage'].str.replace(' km/kg', '', regex=False)
    df['mileage'] = pd.to_numeric(df['mileage'])

    df['engine'] = df['engine'].str.replace(' CC', '', regex=False)
    df['engine'] = pd.to_numeric(df['engine'])


    df['max_power'] = df['max_power'].str.replace(' bhp', '', regex=False)
    df['max_power'] = pd.to_numeric(df['max_power'])
    
    df = df.dropna(how='any')
    return df

df_train = pre_process(df_train)
df_test = pre_process(df_test)

print(df_samples.head())

                                    name  year  selling_price  km_driven  \
0     Ford Figo Aspire 1.5 TDCi Titanium  2017         670000      70000   
1  Mahindra Scorpio VLX 2WD AIRBAG BSIII  2012         525000     120000   
2                 Maruti Swift Dzire VDI  2014         438999      81000   
3              Ford Figo Diesel Titanium  2010         144000      50000   
4                 Hyundai i10 Magna 1.1L  2008         185000     110000   

     fuel seller_type transmission                 owner     mileage   engine  \
0  Diesel  Individual       Manual           First Owner  25.83 kmpl  1498 CC   
1  Diesel  Individual       Manual           First Owner  12.05 kmpl  2179 CC   
2  Diesel      Dealer       Manual           First Owner   23.4 kmpl  1248 CC   
3  Diesel  Individual       Manual          Second Owner   20.0 kmpl  1399 CC   
4  Petrol  Individual       Manual  Fourth & Above Owner  19.81 kmpl  1086 CC   

   max_power               torque  seats prediction     

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

X_train = df_train.drop('selling_price', axis=1)
y_train = df_train['selling_price']
X_test = df_test.drop('selling_price', axis=1)
y_test = df_test['selling_price']

categorical_features = ['name', 'fuel', 'seller_type', 'transmission', 'owner', 'torque']

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features)],
    remainder='passthrough'  
)

# Define your polynomial degree
degree = 2

# Pipeline for polynomial features and regression
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures(degree=degree, include_bias=False)),
    ('regressor', LinearRegression())
])

pipeline.fit(X_train, y_train)

y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Training RMSE: {train_rmse}")
print(f"Test RMSE: {test_rmse}")
print(f"Training R^2: {train_r2}")
print(f"Test R^2: {test_r2}")



Training RMSE: 634206.3971366704
Test RMSE: 555272.5737805899
Training R^2: 0.3953902472002565
Test R^2: 0.41134975432193954


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [16]:
df_samples['prediction'] = y_test_pred

In [17]:
print(df_samples.head())

                                    name  year  selling_price  km_driven  \
0     Ford Figo Aspire 1.5 TDCi Titanium  2017         670000      70000   
1  Mahindra Scorpio VLX 2WD AIRBAG BSIII  2012         525000     120000   
2                 Maruti Swift Dzire VDI  2014         438999      81000   
3              Ford Figo Diesel Titanium  2010         144000      50000   
4                 Hyundai i10 Magna 1.1L  2008         185000     110000   

     fuel seller_type transmission                 owner     mileage   engine  \
0  Diesel  Individual       Manual           First Owner  25.83 kmpl  1498 CC   
1  Diesel  Individual       Manual           First Owner  12.05 kmpl  2179 CC   
2  Diesel      Dealer       Manual           First Owner   23.4 kmpl  1248 CC   
3  Diesel  Individual       Manual          Second Owner   20.0 kmpl  1399 CC   
4  Petrol  Individual       Manual  Fourth & Above Owner  19.81 kmpl  1086 CC   

   max_power               torque  seats     prediction 

In [19]:
df_samples.to_csv(OUTPUT_PATH, index=False)


In [20]:
import numpy as np

all_features = X_train.columns
numerical_features = [col for col in all_features if col not in categorical_features]

onehot_features = preprocessor.named_transformers_['onehot'].get_feature_names_out(categorical_features)
transformed_feature_names = np.concatenate([onehot_features, numerical_features])

lasso_coefficients = pipeline.coef_

feature_coefficients = dict(zip(transformed_feature_names, lasso_coefficients))

numerical_feature_coefficients = {feature: coef for feature, coef in feature_coefficients.items() if feature in numerical_features}

print("Numerical feature coefficients from Lasso Regression:")
for feature, coef in numerical_feature_coefficients.items():
    print(f"{feature}: {coef}")


AttributeError: 'Pipeline' object has no attribute 'coef_'

In [13]:

lasso_coefficients = model.coef_

onehot_feature_names = preprocessor.named_transformers_['onehot'].get_feature_names_out(categorical_features)

onehot_feature_coefficients = dict(zip(onehot_feature_names, lasso_coefficients[:len(onehot_feature_names)]))

top_weights_per_categorical_feature = {}
for original_feature in categorical_features:
    current_feature_coeffs = {feature: coef for feature, coef in onehot_feature_coefficients.items() if feature.startswith(original_feature)}
    
    top_features = sorted(current_feature_coeffs.items(), key=lambda x: abs(x[1]), reverse=True)[:5]
    
    top_weights_per_categorical_feature[original_feature] = top_features

for feature, weights in top_weights_per_categorical_feature.items():
    print(f"Top 5 weights for {feature}:")
    for weight in weights:
        print(f"  {weight[0]}: {weight[1]}")
    print("") 


Top 5 weights for name:
  name_Volvo XC90 T8 Excellence BSIV: 7886940.749389883
  name_Audi A6 35 TFSI Matrix: 4219838.68796118
  name_Mercedes-Benz S-Class S 350 CDI: 4085709.416624769
  name_Volvo XC60 Inscription D5 BSIV: 4009573.143621004
  name_BMW 6 Series GT 630d Luxury Line: 3978452.1546066776

Top 5 weights for fuel:
  fuel_Diesel: 94793.5538955685
  fuel_LPG: 53554.118327293734
  fuel_CNG: -13293.74598238184
  fuel_Petrol: -402.5393102986331

Top 5 weights for seller_type:
  seller_type_Trustmark Dealer: -81804.9098904356
  seller_type_Dealer: 22140.23830742119
  seller_type_Individual: 11514.651168594737

Top 5 weights for transmission:
  transmission_Automatic: 156371.7607120357
  transmission_Manual: -1.1126889289197336e-06

Top 5 weights for owner:
  owner_Test Drive Car: 536738.9785793227
  owner_First Owner: 27776.354307328493
  owner_Third Owner: -8623.491065852064
  owner_Fourth & Above Owner: -23.650747973642964
  owner_Second Owner: -0.0

Top 5 weights for torque:
 